# Sanity Checking the Subspace Network Digital Twin

This notebook performs an simulation run that uses the default parameter set
for a single run. It is meant to provide quick diagnostics on the model
capabilities

In [1]:
import sys
sys.path.append('../')

import pandas as pd
import os
import plotly.express as px

In [2]:
import pandas as pd
import plotly.express as px
import matplotlib.pyplot as plt
import numpy as np
# %%
from subspace_model.params import INITIAL_STATE
from subspace_model.params import SINGLE_RUN_PARAMS
from subspace_model.params import TIMESTEPS
from subspace_model.params import SAMPLES


from subspace_model.structure import SUBSPACE_MODEL_BLOCKS
from cadCAD_tools import easy_run

default_run_args = (INITIAL_STATE,
                     {k: [v] for k, v in SINGLE_RUN_PARAMS.items()},
                    SUBSPACE_MODEL_BLOCKS,
                    TIMESTEPS,
                    SAMPLES)
sim_df = easy_run(*default_run_args)


                  ___________    ____
  ________ __ ___/ / ____/   |  / __ \
 / ___/ __` / __  / /   / /| | / / / /
/ /__/ /_/ / /_/ / /___/ ___ |/ /_/ /
\___/\__,_/\__,_/\____/_/  |_/_____/
by cadCAD

cadCAD Version: 0.4.28
Execution Mode: local_proc
Simulation Dimensions:
Entire Simulation: (Models, Unique Timesteps, Params, Total Runs, Sub-States) = (1, 701, 12, 1, 22)
     Simulation 0: (Timesteps, Params, Runs, Sub-States) = (701, 12, 1, 22)


Initializing configurations:   0%|          | 0/1 [00:00<?, ?it/s]

Execution Method: local_simulations
Execution Mode: single_threaded


Flattening results:   0%|          | 0/7712 [00:00<?, ?it/s]

Total execution time: 0.35s


In [3]:
sim_df.head(5)

,days_passed,delta_days,reward_issuance_balance,other_issuance_balance,operators_balance,nominators_balance,holders_balance,farmers_balance,staking_pool_balance,fund_balance,...,sector_size_in_bytes,block_time_in_seconds,archival_duration_in_blocks,archive_size_in_bytes,reward_proposer_share,max_credit_supply,fund_tax_on_proposer_reward,fund_tax_on_storage_fees,farmer_tax_on_compute_priority_fees,operator_tax_on_compute_revenue
0,0,0,1.320000e+09,1.680000e+09,0.000000e+00,0.000000e+00,0.0,0.000000e+00,0.000000e+00,0.0,...,32768000,6,100,128000000.0,0.0,3000000000,0.0,0.0,0.0,0.0
11,1,1,1.306800e+09,1.680000e+09,1.142883e+06,1.142883e+06,0.0,1.320004e+07,2.285766e+06,0.0,...,32768000,6,100,128000000.0,0.0,3000000000,0.0,0.0,0.0,0.0
22,2,1,1.293732e+09,1.680000e+09,1.067000e+06,1.067000e+06,0.0,2.626807e+07,4.419765e+06,0.0,...,32768000,6,100,128000000.0,0.0,3000000000,0.0,0.0,0.0,0.0
33,3,1,1.280795e+09,1.680000e+09,9.454741e+05,9.454741e+05,0.0,3.920542e+07,6.310714e+06,0.0,...,32768000,6,100,128000000.0,0.0,3000000000,0.0,0.0,0.0,0.0
44,4,1,1.267987e+09,1.680000e+09,8.565138e+05,8.565138e+05,0.0,5.201338e+07,8.023741e+06,0.0,...,32768000,6,100,128000000.0,0.0,3000000000,0.0,0.0,0.0,0.0


In [6]:
px.line(sim_df, x='days_passed', y=[el for el in sim_df.columns if "_balance" in el],
        log_y=True)

In [8]:
px.line(sim_df, x='days_passed', y=[el for el in sim_df.columns if "volume" in el],
        log_y=True)

In [9]:
px.line(sim_df, x='days_passed', y=[el for el in sim_df.columns if "average" in el],
        log_y=True)